In [4]:
!pip install graphviz
!pip install web3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 1.9 MB/s             


In [2]:
from tree import Node, insert, vis

root = Node(1)
for key in range(2, 32):
    insert(root, key)

vis(root)

In [12]:
from block import Block
from game import FaultDisputeGame
from pprint import pprint
import json
from web3 import Web3

ABSOLUTE_PRESTATE = "0"
MAX_GAME_DEPTH = 73
SPLIT_DEPTH = 30
MAX_CLOCK_DURATION = 3.5 * 24 * 3600
CLOCK_EXTENSION = 3 * 3600
CHALLENGE_PERIOD = 24 * 3600

def hash_str(s: str) -> str:
    vals = [s]
    types = ['string']
    # keccak256(abi.encodePacked(...))
    return Web3.solidity_keccak(types, vals).hex()

block = Block()
game = FaultDisputeGame()
game.initialize(
    l2_root = hash_str("l2 root"),
    l2_block_num = 1,
    root_claim = hash_str("root"),
    game_creator = "defender",
    bond = 1,
    block_timestamp = block.timestamp
)

###  no challenge -> defender wins ###
# block.warp(MAX_CLOCK_DURATION)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp
# game.resolve()
# game.status

### attack -> challenger wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# # Each party (defender and challenger) gets MAX_CLOCK_DURATION
# # Need to wait for both parties to use MAX_CLOCK_DURATION
# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defender wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = hash_str("2,0"), 
#     challenge_idx = 1, 
#     claim = hash_str("6,0"), 
#     is_attack = False, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> attack -> defender wins ###
# block.inc()
# game.move(
#     disputed = hash_str("root"), 
#     challenge_idx = 0, 
#     claim = hash_str("2,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "challenger",
# )

# block.inc()
# game.move(
#     disputed = hash_str("2,0"), 
#     challenge_idx = 1, 
#     claim = hash_str("4,0"), 
#     is_attack = True, 
#     msg_value = 0, 
#     block_timestamp = block.timestamp,
#     msg_sender = "defender",
# )

# block.warp(MAX_CLOCK_DURATION + 1)
# game.resolve_claim(2, 0, block_timestamp = block.timestamp)
# game.resolve_claim(1, 0, block_timestamp = block.timestamp)
# game.resolve_claim(0, 0, block_timestamp = block.timestamp)
# game.resolve()
# game.status

### attack -> defend -> defend -> challenger wins ###
block.inc()
game.move(
    disputed = hash_str("root"), 
    challenge_idx = 0, 
    claim = hash_str("2,0"), 
    is_attack = True, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.inc()
game.move(
    disputed = hash_str("2,0"), 
    challenge_idx = 1, 
    claim = hash_str("6,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "defender",
)

block.inc()
game.move(
    disputed = hash_str("6,0"), 
    challenge_idx = 2, 
    claim = hash_str("14,0"), 
    is_attack = False, 
    msg_value = 0, 
    block_timestamp = block.timestamp,
    msg_sender = "challenger",
)

block.warp(MAX_CLOCK_DURATION + 2)
game.resolve_claim(3, 0, block_timestamp = block.timestamp)
game.resolve_claim(2, 0, block_timestamp = block.timestamp)
game.resolve_claim(1, 0, block_timestamp = block.timestamp)
game.resolve_claim(0, 0, block_timestamp = block.timestamp)
game.resolve()
game.status

claim idx 0 claim timestamp 0 block 1 parent dur 0 challenge dur 1
claim idx 1 claim timestamp 1 block 2 parent dur 0 challenge dur 1
claim idx 2 claim timestamp 2 block 3 parent dur 1 challenge dur 2
claim idx 3 claim timestamp 3 block 302402.0 parent dur 1 challenge dur 302400.0
302400
claim idx 2 claim timestamp 2 block 302402.0 parent dur 1 challenge dur 302401.0
302400
claim idx 1 claim timestamp 1 block 302402.0 parent dur 0 challenge dur 302401.0
302400
claim idx 0 claim timestamp 0 block 302402.0 parent dur 0 challenge dur 302402.0
302400


<GameStatus.CHALLENGER_WINS: 1>

In [2]:
from game_types import VMStatus

VMStatus(1)

<VMStatus.INVALID: 1>